In [1]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.manager import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

class Suite2pROICheckGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_CHECK"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager = initManagers(WidgetManager(), ConfigManager(), DataManager(), ControlManager())
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]
        self.config_manager.setDictTableColumns(key_app=self.app_key_pri)

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        self.initUI()

    """
    setup UI Function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)

        # FileLoadUI用のレイアウト
        self.layout_file_load = QVBoxLayout()
        self.setupFileLoadUI()
        self.layout_main.addLayout(self.layout_file_load, 1, 0, 1, 1)

        # メインUI用のレイアウト
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui, 0, 0, 1, 1)

    def setupFileLoadUI(self):
        file_load_widget = QWidget()
        layout = QVBoxLayout(file_load_widget)
        # ファイル読み込み用のUIを追加
        layout.addLayout(self.makeLayoutSectionBottom())
        # bindFunc
        self.bindFuncFileLoadUI()

        self.layout_file_load.addWidget(file_load_widget)

    def loadFilePathsandInitialize(self):
        self.control_manager, self.data_manager = initManagers(self.control_manager, self.data_manager)
        success = self.loadData()
        if success:
            self.setupMainUI()
        else:
            return

    def setupMainUI(self):
        if self.setupUI_done:
            # メインUIのクリア
            clearLayout(self.layout_main_ui)
        
        # 新しいメインUIの設定
        self.setupMainUILayouts()
        self.setupControls()
        self.bindFuncAllWidget()

        self.setupUI_done = True

    def loadData(self):
        success = loadFallMATWithGUI(
            q_window=self, 
            data_manager=self.data_manager, 
            key_app=self.app_key_pri, 
            path_fall=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_fall"].text()
        )
        if self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text() != "":
            self.data_manager.dict_im_bg_chan2[self.app_key_pri] = loadTIFImage(
                data_manager=self.data_manager, 
                key_dict_im_chan2=self.app_key_pri, 
                path_image=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_reftif"].text(), 
                preprocessing=True
                )
        return success

    def setupMainUILayouts(self):
        self.layout_main_ui.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0)
        self.layout_main_ui.addLayout(self.makeLayoutSectionMiddleUpper(), 0, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionRightUpper(), 0, 2)

    def setupControls(self):
        self.widget_manager.dict_table[self.app_key_pri] = setupWidgetROITable(
                                                                        q_table=self.widget_manager.dict_table[self.app_key_pri], 
                                                                        len_row=len(self.data_manager.dict_Fall[self.app_key_pri]["F"]), 
                                                                        dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]
                                                                    )
        self.control_manager.table_controls[self.app_key_pri] = TableControls(
                                                                key_app=self.app_key_pri,
                                                                q_table=self.widget_manager.dict_table[self.app_key_pri],
                                                                data_manager=self.data_manager,
                                                                widget_manager=self.widget_manager,
                                                                config_manager=self.config_manager,
                                                                dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri],
                                                                key_function_map=self.config_manager.key_function_map
                                                                )
        self.widget_manager.dict_table[self.app_key_pri].keyPressEvent = lambda event: self.control_manager.table_controls[self.app_key_pri].keyPressEvent(event)
        self.control_manager.view_controls[self.app_key_pri] = ViewControls(
                                                                    key_app=self.app_key_pri,
                                                                    q_view=self.widget_manager.dict_view[self.app_key_pri], 
                                                                    q_scene=self.widget_manager.dict_scene[self.app_key_pri], 
                                                                    data_manager=self.data_manager, 
                                                                    widget_manager=self.widget_manager,
                                                                    config_manager=self.config_manager,
                                                                    )
        self.control_manager.view_controls[self.app_key_pri].setViewSize()

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    # Main Layout
    def makeLayoutMain(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        return QGridLayout(central_widget)

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()

        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)", "Cellpose Mask path (optional)"]
        list_key = [f"{self.app_key_pri}_path_fall", f"{self.app_key_pri}_path_reftif", f"{self.app_key_pri}_path_cellpose"]
        for label, key in zip(list_label, list_key):
            layout.addLayout(makeLayoutLoadFileWidget(self.widget_manager, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    "Left Upper"
    # EventFileの読み込み, plot用
    def makeLayoutComponentEventFilePlot(self):
        layout = QVBoxLayout()
        # Eventのplot range
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                 key_label="eventfile_align_plot_range",
                                                 key_lineedit="eventfile_align_plot_range",
                                                 label="plot range from Event start (pre, post; sec)",
                                                 text_set="(10, 10)"))
        # プロットに重ねるEventFile npyファイルの読み込みボタン, Clearボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(key="loadEventFile", label="Load EventFile npy file"))  ### 要修正
        layout.addWidget(self.widget_manager.makeWidgetButton(key="clearEventFile", label="Clear"))
        return layout

    # trace plotの表示範囲
    def makeLayoutComponentTracePlotRange(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                key_label="minPlotRange",
                                                key_lineedit="minPlotRange",
                                                label="Minimum plot range (sec)",
                                                text_set="30"))
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="plot_eventfile_trace", 
                                                 label="plot EventFile trace", 
                                                 checked=True,
                                                 ))
        return layout

    # 上記二つを合体
    def makeLayoutComponentEventFilePlot_TracePlotRange(self):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentEventFilePlot())
        layout.addLayout(self.makeLayoutComponentTracePlotRange())
        return layout

    "Middle Upper"
    # ROI view
    def makeLayoutComponentROIView(self):
        layout = QVBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetView(key=self.app_key_pri))
        return layout

    # ROI property label, threshold lineedit
    def makeLayoutComponentROIPropertyDisplay_Threshold(self):
        layout = QVBoxLayout()

        layout.addLayout(makeLayoutROIProperty(self.widget_manager, key_label=f"{self.app_key_pri}_roi_prop"))
        layout.addLayout(makeLayoutROIThresholds(
            self.widget_manager, 
            key_label=f"{self.app_key_pri}_roi_threshold", 
            key_lineedit=f"{self.app_key_pri}_roi_threshold", 
            key_checkbox=f"{self.app_key_pri}_roi_threshold", 
            label_checkbox="ROI Show Threshold", 
            list_threshold_param=["npix", "compact"]))
        return layout

    # ROI display, background image button group, checkbox
    def makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutROITypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.app_key_pri}_roi_type', dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]))
        layout.addLayout(makeLayoutBGImageTypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.app_key_pri}_bgimage_type'))
        layout.addLayout(makeLayoutROIChooseSkip(self.widget_manager, key_checkbox=f'{self.app_key_pri}_roi_skip_choose', dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]))
        return layout

    # channel contrast, ROI opacity slider
    def makeLayoutComponentContrastOpacitySlider(self):
        layout = QVBoxLayout()
        channels = self.config_manager.gui_defaults["CHANNELS"]
        layout_channel = QHBoxLayout()
        for channel in channels:
            layout_channel.addLayout(makeLayoutContrastSlider(self.widget_manager, 
                                                            key_label=f"{self.app_key_pri}_{channel}", 
                                                            key_checkbox=f"{self.app_key_pri}_{channel}", 
                                                            key_slider=f"{self.app_key_pri}_{channel}", 
                                                            label_checkbox=f"Show {channel} channel", 
                                                            label_label=f"{channel} Value", 
                                                            checked=True))

        layout.addLayout(layout_channel)
        layout.addLayout(makeLayoutOpacitySlider(self.widget_manager, 
                                                key_label=self.app_key_pri, 
                                                key_slider=self.app_key_pri, 
                                                label=self.app_key_pri
                                                ))
        return layout

    "Right Upper"
    # Table, ROI count label, Set ROI Celltype, ROICheck IO
    def makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutTableROICountLabel(self.widget_manager, key_label=self.app_key_pri, key_table=self.app_key_pri, dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]))
        layout.addLayout(makeLayoutAllROISetSameCelltype(self.widget_manager, key_button=self.app_key_pri, dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]))
        layout.addLayout(makeLayoutROICheckIO(self.widget_manager, key_button=self.app_key_pri))
        return layout

    # ROI Filter, threshold
    def makeLayoutComponentROIFilter(self):
        layout = QHBoxLayout()
        layout.addLayout(makeLayoutROIFilterThreshold(self.widget_manager, key_label=self.app_key_pri, key_lineedit=self.app_key_pri, dict_roi_threshold=self.config_manager.gui_defaults["ROI_THRESHOLDS"]))
        layout.addLayout(makeLayoutROIFilterButton(self.widget_manager, key_label=self.app_key_pri, key_button=self.app_key_pri))
        return layout
    

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 左上
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutCanvasTracePlot(self.widget_manager, 
                                                   key_figure=self.app_key_pri, 
                                                   key_canvas=self.app_key_pri, 
                                                   key_app=self.app_key_pri), 
                        stretch=1)
        layout.addLayout(self.makeLayoutComponentEventFilePlot_TracePlotRange())
        return layout

    # 中上
    def makeLayoutSectionMiddleUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIView())
        layout.addLayout(self.makeLayoutComponentROIPropertyDisplay_Threshold())
        layout.addLayout(self.makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip())
        layout.addLayout(self.makeLayoutComponentContrastOpacitySlider())
        return layout
    
    # 右上
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO())
        layout.addLayout(self.makeLayoutComponentROIFilter())
        return layout

    # 下
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):        
        list_key = [f"{self.app_key_pri}_path_fall", f"{self.app_key_pri}_path_reftif", f"{self.app_key_pri}_path_cellpose"]
        list_filetype = ["mat", "tiff", "npy"]
        for key, filetype in zip(list_key, list_filetype):
            bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.loadFilePathsandInitialize())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        # ROICheck save load
        bindFuncROICheckIO(
            q_window=self, 
            q_lineedit=self.widget_manager.dict_lineedit[f"{self.app_key_pri}_path_fall"], 
            q_table=self.widget_manager.dict_table[f"{self.app_key_pri}"], 
            q_button_save=self.widget_manager.dict_button[f"{self.app_key_pri}_save_roicheck"], 
            q_button_load=self.widget_manager.dict_button[f"{self.app_key_pri}_load_roicheck"], 
            dict_tablecol=self.config_manager.dict_tablecol[self.app_key_pri]
        )
        # Radiobutton BGImageType buttonClicked
        bindFuncRadiobuttonBGImageTypeChanged(
            q_buttongroup=self.widget_manager.dict_buttongroup[f"{self.app_key_pri}_bgimage_type"], 
            data_manager=self.data_manager,
            view_controls=self.control_manager.view_controls[self.app_key_pri],
            key_app=self.app_key_pri
        )
        # ROICheck Table onSelectionChanged
        bindFuncTableSelectionChanged(
            q_table=self.widget_manager.dict_table[self.app_key_pri],
            data_manager=self.data_manager,
            view_controls=self.control_manager.view_controls[self.app_key_pri],
            key_app=self.app_key_pri,
        )
        # Slider Opacity valueChanged
        bindFuncOpacitySlider(
            q_slider=self.widget_manager.dict_slider[f"{self.app_key_pri}_opacity_allroi"],
            view_controls=self.control_manager.view_controls[self.app_key_pri],
        )
        bindFuncHighlightOpacitySlider(
            q_slider=self.widget_manager.dict_slider[f"{self.app_key_pri}_opacity_selectedroi"],
            view_controls=self.control_manager.view_controls[self.app_key_pri],
        )
        # Slider Contrast valueChanged, Checkbox show channel stateChanged
        for channel in self.config_manager.gui_defaults["CHANNELS"]:
            bindFuncBackgroundContrastSlider(
                q_slider_min=self.widget_manager.dict_slider[f"{self.app_key_pri}_{channel}_contrast_min"],
                q_slider_max=self.widget_manager.dict_slider[f"{self.app_key_pri}_{channel}_contrast_max"],
                view_controls=self.control_manager.view_controls[self.app_key_pri],
                channel=channel
            )
            bindFuncBackgroundVisibilityCheckbox(
                q_checkbox=self.widget_manager.dict_checkbox[f"{self.app_key_pri}_{channel}_show"], 
                view_controls=self.control_manager.view_controls[self.app_key_pri],
                channel=channel,
            )


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

TypeError: highlightSelectedROI() missing 1 required positional argument: 'key_app'

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
gui.control_manager.view_controls

{}

In [2]:
gui.data_manager.dict_im_bg_chan2

{}